In [1]:
import cv2
import mediapipe as mp
import pyautogui

In [2]:
# Create a hand_tracker object using mp
hand_tracker = mp.solutions.hands.Hands(max_num_hands = 1, model_complexity = 0)

# Create a drawind_utils object that draws the dots identified by hand markers
drawing_utils = mp.solutions.drawing_utils

# Getting the size of the screen using pyautogui
screen_width, screen_height = pyautogui.size()

# Setting the failsafe object as false to avoid the cursor moving out of the screen failure
pyautogui.FAILSAFE = False

# Initializing two arrays that contain the landmark coordinates and assigning them initial value 0
x0 = x4 = x5 = x8 = x9 = x12 = x17 = y0 = y4 = y5 = y8 = y9 = y12 = y17 = 0.0
old_x8 = screen_width / 2.0
old_y8 = screen_height / 2.0 

In [3]:
# Function to adjust the coordinate of landmark with the screen size
def adjusted_coord(x, y): 
    adjusted_x = screen_width*x
    adjusted_y = screen_height*y
    return adjusted_x, adjusted_y

In [4]:
def move_cursor(landmarks):
    global old_x8, old_y8
    # Adjusting the frame dimensions to screen dimensions for moving the cursor
    x8, y8 = adjusted_coord(landmarks[8].x, landmarks[8].y)
    
    # storing the difference between the position of current coord and previous coord
    # We are also rescaling the value in order to manage the sensitivity
    if abs(x8 - old_x8) > 100 or abs(y8 - old_y8) > 100:
        diff_x8 = 0
        diff_y8 = 0
    else:
        diff_x8 = int((x8 - old_x8)*1.50)
        diff_y8 = int((y8 - old_y8)*1.50)
            
    # Changing the value of old coordinates to current coordinates
    old_x8 = x8
    old_y8 = y8
        
    pyautogui.move(diff_x8, diff_y8)

In [6]:
# Capture the video 
cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)

# Set the window size 
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1500)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 1500)

# Capturing each frame from the camera
while True:
    ret, frame = cap.read()
    
    # If there is no feed then break the loop
    if not ret:
        break
    
    frame_height, frame_width, _ = frame.shape
    
    # Flip the frame
    frame = cv2.flip(frame, 1)
    
    # We need to convert the frame from bgr to rgb so that we can work it out in media pipe as well
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Now we detect the hand using the hand_tracker object we created
    output = hand_tracker.process(frame_rgb)
    hands = output.multi_hand_landmarks
    
    # Check if there is anything in the hands variable and then draw them
    if hands:

        drawing_utils.draw_landmarks(frame, hands[0])
            
        # Landmarks of the hand
        landmarks = hands[0].landmark
        
        # We are only going to move the cursor when the hand makes a V shape.
        # To check the "V" shape we are simply ensuring that the tip of thumb, ring finger and pinky are close enough
        x4, y4 = adjusted_coord(landmarks[4].x, landmarks[4].y)
        x16, y16 = adjusted_coord(landmarks[16].x, landmarks[16].y)
        x20, y20 = adjusted_coord(landmarks[20].x, landmarks[20].y)
        
        if abs(x4-x16)<50 and abs(x4-x20)<50:
            move_cursor(landmarks)
        
        # Left Click if the tip of the index finger is close enough to the base of the index finger.
        # Same logic for right click with the middle finger
        x8, y8 = adjusted_coord(landmarks[8].x, landmarks[8].y)
        x12, y12 = adjusted_coord(landmarks[12].x, landmarks[12].y)
        x5, y5 = adjusted_coord(landmarks[5].x, landmarks[5].y)
        x9, y9 = adjusted_coord(landmarks[9].x, landmarks[9].y)
        
        if abs(y8 - y5) < 75:
            pyautogui.click(button = 'left')
            pyautogui.sleep(0.5)
                        
        if abs(y12 - y9) < 75:
            pyautogui.click(button = 'right')
            pyautogui.sleep(0.5)
            
        
    # Display the captured frame 
    cv2.imshow("Captured Video", frame)
    
    # Exiting the loop when escape is pressed
    key = cv2.waitKey(1)
    if key == 27:
        break
        
# Release the capture and destroy all windows
cap.release()
cv2.destroyAllWindows()

### Other Similar functions like scroll up/down, drag and drop and other functions as and if necessary by using a similar approach as the rest.